In [1]:
import psycopg2
from psycopg2.extensions import AsIs
import sqlite3, pdb
import pandas as pd
import numpy as np

In [2]:
#dft1 = pd.read_csv('../ncpfast-db/analytes.csv', sep='|')
#dft1.head()

In [2]:
def getVariablesT1():
    try:
        conn = sqlite3.connect('../RoundOneDB/RoundOneDB_final.db')
        cur = conn.cursor()
        
        cur.execute("""SELECT DISTINCT analyte_name, abbreviation, cas, class
                       FROM quant_summary 
                       ORDER BY analyte_name""")
        rows = cur.fetchall()
        cur.close()
        conn.close()
        
        df = pd.DataFrame(columns = ['analyte_name','abbreviation','cas','class']) 
        
        for row in rows:
            df = df.append({'analyte_name': row[0],'abbreviation': row[1],'cas': row[2], 'class': row[3]},
                            ignore_index = True)

        return df
    
    except Error as e:
        print(e)

In [3]:
dft1 = getVariablesT1()
dft1.head()

,analyte_name,abbreviation,cas,class
0,"1,1,2,2-tetrafluoro-2-(1,2,2,2-tetrafluoro-eth...",NVHOS,N/A,Perfluoroalkylether Acids (PFEAs)
1,10:2 Fluorotelomer sulfonic acid,10:2 PFS,120226-60-0,Fluorotelomer Sulfonates
2,11-chloroeicosafluoro-3-oxanonane-1-sulfonate,F-53B (Minor),83329-89-9,Perfluoroalkylether Acids (PFEAs)
3,4:2 Fluorotelomer sulfonic acid,4:2 PFS,757124-72-4,Fluorotelomer Sulfonates
4,6:2 Fluorotelomer sulfonic acid,6:2 PFS,27619-97-2,Fluorotelomer Sulfonates


In [5]:
def getVariablesCV():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT name, term FROM cv_variable_names ORDER BY name""")
        rows = cur.fetchall()
        
        df = pd.DataFrame(columns = ['analyte_name','cas']) 
        
        for row in rows:
            df = df.append({'analyte_name': row[0],'cas': row[1]}, 
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [6]:
dfdb = getVariablesCV()
dfdb.head()

,analyte_name,cas
0,"10,10-Diethoxy-1,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8...",133377-48-7
1,10:1 Fluorotelomer alcohol,307-46-0
2,10:2 Fluorotelomer acrylate,17741-60-5
3,10:2 Fluorotelomer alcohol,865-86-1
4,10:2 Fluorotelomer iodide,2043-54-1


In [7]:
caslist = list(dfdb['cas'])

analytes = []

dfyes = pd.DataFrame(columns = ['status','analyte_name','cas'])
dfno = pd.DataFrame(columns = ['status','analyte_name','cas'])

for index, row in dft1.iterrows():
    if row[2] in caslist:
        dfyes = dfyes.append({'status': 'YES','analyte_name': str(row[0]),'cas': str(row[2])},
                              ignore_index = True)
    if row[0] == '1,1,2,2-tetrafluoro-2-(1,2,2,2-tetrafluoro-ethoxy)ethane sulfonic acid':
        dfyes = dfyes.append({'status': 'YES','analyte_name': str(row[0]),'cas': '801209-99-4'},
                              ignore_index = True)        
    elif row[2] not in caslist:
        dfno = dfno.append({'status': 'NO','analyte_name': str(row[0]),'cas': str(row[2])}, 
                              ignore_index = True)


In [27]:
dfyes = pd.read_csv('../ncpfast-db/analyte_cv_yes.csv')      
dfyes

,status,analyte_name,cas
0,YES,"1,1,2,2-tetrafluoro-2-(1,2,2,2-tetrafluoro-eth...",801209-99-4
1,YES,"Ethanesulfonic acid, 2-[1-[difluoro(1,2,2,2-te...",749836-20-2
2,YES,"Ethanesulfonic acid, 2-[1-[difluoro[(1,2,2-tri...",29311-67-9
3,YES,"Perfluoro(3,5,7,9,11-pentaoxadodecanoic) acid",39492-91-6
4,YES,"Perfluoro(3,5,7,9-tetraoxadecanoic) acid",39492-90-5
5,YES,"Perfluoro(3,5,7-trioxaoctanoic) acid",39492-89-2
6,YES,"Perfluoro(3,5-dioxahexanoic) acid",39492-88-1
7,YES,Perfluoro-2-ethoxypropanoic acid,267239-61-2
8,YES,Perfluoro-2-methoxyacetic acid,674-13-5
9,YES,Perfluoro-2-propoxypropanoic acid,13252-13-6


In [4]:
dfno = pd.read_csv('../ncpfast-db/analyte_cv_no.csv')
dfno

,status,analyte_name,cas
0,NO,10:2 Fluorotelomer sulfonic acid,120226-60-0
1,NO,11-chloroeicosafluoro-3-oxanonane-1-sulfonate,83329-89-9
2,NO,4:2 Fluorotelomer sulfonic acid,757124-72-4
3,NO,6:2 Fluorotelomer sulfonic acid,27619-97-2
4,NO,8:2 Fluorotelomer sulfonic acid,39108-34-4
5,NO,9-chlorohexadecafluoro-3-oxanonane-1-sulfonate,73606-19-6
6,NO,"Dodecafluoro-3H-4,8-dioxanonanoic acid",958445-44-8
7,NO,N-(3-dimethylaminopropan-1-yl)perfluoro-1-hexa...,50598-28-2
8,NO,"N-(carboxymethyl)-N,N-dimethyl-N-[3-(1H,1H,2H,...",34455-29-3
9,NO,N-[3-(perfluoro-1-hexanesulfonamido)propan-1-y...,38850-51-0


In [5]:
casno = list(dfno['cas'])
dfinsert = pd.DataFrame(columns = ['analyte_name','cas','abbreviation'])

for index, row in dft1.iterrows():
    if row[2] in casno:
        dfinsert = dfinsert.append({'analyte_name': str(row[0]),'cas': str(row[2]),'abbreviation': row[1]},
                                 ignore_index = True)
dfinsert

,analyte_name,cas,abbreviation
0,10:2 Fluorotelomer sulfonic acid,120226-60-0,10:2 PFS
1,11-chloroeicosafluoro-3-oxanonane-1-sulfonate,83329-89-9,F-53B (Minor)
2,4:2 Fluorotelomer sulfonic acid,757124-72-4,4:2 PFS
3,6:2 Fluorotelomer sulfonic acid,27619-97-2,6:2 PFS
4,8:2 Fluorotelomer sulfonic acid,39108-34-4,8:2 PFS
5,9-chlorohexadecafluoro-3-oxanonane-1-sulfonate,73606-19-6,F-53B (Major)
6,"Dodecafluoro-3H-4,8-dioxanonanoic acid",958445-44-8,ADONA
7,N-(3-dimethylaminopropan-1-yl)perfluoro-1-hexa...,50598-28-2,N-AP-FHxSA
8,"N-(carboxymethyl)-N,N-dimethyl-N-[3-(1H,1H,2H,...",34455-29-3,N-CMAmP-6:2FOSA (6:2 FTAB)
9,N-[3-(perfluoro-1-hexanesulfonamido)propan-1-y...,38850-51-0,N-TAmP-FHxSA


In [6]:
dfinsert.to_csv('../ncpfast-db/analyte_cv_insert.csv', index=False)

In [10]:
#dfyes.to_csv('../ncpfast-db/analyte_cv_yes.csv', index=False)
#dfno.to_csv('../ncpfast-db/analyte_cv_no.csv', index=False)

In [11]:
def crossRefVariables(dfyes):
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        
        df = pd.DataFrame(columns = ['analyte_name1', 'analyte_name2','casnum1', 'casnum2']) 
        
        for index, row in dfyes.iterrows():
            if row['status'] == 'YES':
                casnum = row['cas']
                cur.execute("""SELECT term, name, definition FROM cv_variable_names where term = %(cas_num)s""",
                            {'cas_num': casnum})
                rowdb = cur.fetchall()
        
                df = df.append({'analyte_name1': row[1], 'analyte_name2': rowdb[0][1],
                                'casnum1': row[2],'casnum2':rowdb[0][0]}, 
                                ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [12]:
dfref = crossRefVariables(dfyes)
dfref

,analyte_name1,analyte_name2,casnum1,casnum2
0,"1,1,2,2-tetrafluoro-2-(1,2,2,2-tetrafluoro-eth...","1,1,2,2-tetrafluoro-2-(1,2,2,2-tetrafluoro-eth...",801209-99-4,801209-99-4
1,"Ethanesulfonic acid, 2-[1-[difluoro(1,2,2,2-te...",Perfluoro-2-{[perfluoro-3-(perfluoroethoxy)-2-...,749836-20-2,749836-20-2
2,"Ethanesulfonic acid, 2-[1-[difluoro[(1,2,2-tri...","Perfluoro-3,6-dioxa-4-methyl-7-octene-1-sulfon...",29311-67-9,29311-67-9
3,"Perfluoro(3,5,7,9,11-pentaoxadodecanoic) acid","Perfluoro-3,5,7,9,11-pentaoxadodecanoic acid",39492-91-6,39492-91-6
4,"Perfluoro(3,5,7,9-tetraoxadecanoic) acid","Perﬂuoro-3,5,7,9-butaoxadecanoic acid",39492-90-5,39492-90-5
5,"Perfluoro(3,5,7-trioxaoctanoic) acid","Perfluoro-3,5,7-trioxaoctanoic acid",39492-89-2,39492-89-2
6,"Perfluoro(3,5-dioxahexanoic) acid","Perﬂuoro-3,5-dioxahexanoic acid",39492-88-1,39492-88-1
7,Perfluoro-2-ethoxypropanoic acid,"2,3,3,3-Tetrafluoro-2-(pentafluoroethoxy)propa...",267239-61-2,267239-61-2
8,Perfluoro-2-methoxyacetic acid,Difluoro(perfluoromethoxy)acetic acid,674-13-5,674-13-5
9,Perfluoro-2-propoxypropanoic acid,Perfluoro-2-methyl-3-oxahexanoic acid,13252-13-6,13252-13-6


In [13]:
dfref.to_csv('../ncpfast-db/analyte_cross_ref.csv', index=False)

In [3]:
dfbad = pd.read_csv('../ncpfast-db/analyte_cross_ref_no.csv', sep=',')
dfbad

,analyte_name1,analyte_name2,casnum1,casnum2
0,"Ethanesulfonic acid, 2-[1-[difluoro(1,2,2,2-te...",Perfluoro-2-{[perfluoro-3-(perfluoroethoxy)-2-...,749836-20-2,749836-20-2
1,"Ethanesulfonic acid, 2-[1-[difluoro[(1,2,2-tri...","Perfluoro-3,6-dioxa-4-methyl-7-octene-1-sulfon...",29311-67-9,29311-67-9
2,Perfluoro-2-ethoxypropanoic acid,"2,3,3,3-Tetrafluoro-2-(pentafluoroethoxy)propa...",267239-61-2,267239-61-2
3,Perfluoro-2-methoxyacetic acid,Difluoro(perfluoromethoxy)acetic acid,674-13-5,674-13-5
4,Perfluoro-2-propoxypropanoic acid,Perfluoro-2-methyl-3-oxahexanoic acid,13252-13-6,13252-13-6
5,"Propanoic acid, 3-[1-[difluoro(1,2,2,2-tetrafl...","2,2,3,3-Tetrafluoro-3-{[1,1,1,2,3,3-hexafluoro...",773804-62-9,773804-62-9


In [15]:
def checkVariablesDB(dfbad):
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        
        df = pd.DataFrame(columns = ['variableid', 'variablecode', 'nodatavalue', 'variablenamecv', 'term']) 
        
        for index, row in dfbad.iterrows():
            casnum = row['casnum1']
            cur.execute("""SELECT DISTINCT v.variableid, v.variablecode, v.nodatavalue, v.variablenamecv, cv.term
                           FROM results r 
                           INNER JOIN feature_actions fa ON r.featureactionid=fa.featureactionid 
                           INNER JOIN sampling_features sf ON fa.samplingfeatureid=sf.samplingfeatureid
                           INNER JOIN variables v ON r.variableid=v.variableid
                           INNER JOIN cv_variable_names cv ON v.variablenamecv=cv.name
                           WHERE cv.term = %(cas_num)s""",
                       {'cas_num': casnum})
            rowdbs = cur.fetchall()
        
        
            for rowdb in rowdbs:
                df = df.append({'variableid': rowdb[0], 'variablecode': rowdb[1], 'nodatavalue': rowdb[2], 
                                'variablenamecv': rowdb[3], 'term': rowdb[4]}, 
                                ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()

In [16]:
dfx = checkVariablesDB(dfbad)
dfx

,variableid,variablecode,nodatavalue,variablenamecv,term
0,53556,Nafion Bp 2,4.9,Perfluoro-2-{[perfluoro-3-(perfluoroethoxy)-2-...,749836-20-2
1,53555,Nafion Bp 1,4.9,"Perfluoro-3,6-dioxa-4-methyl-7-octene-1-sulfon...",29311-67-9
2,53560,PEPA,4.9,"2,3,3,3-Tetrafluoro-2-(pentafluoroethoxy)propa...",267239-61-2
3,53558,PFMOAA,4.9,Difluoro(perfluoromethoxy)acetic acid,674-13-5
4,53561,GenX,4.9,Perfluoro-2-methyl-3-oxahexanoic acid,13252-13-6
5,53566,HydroEve,4.9,"2,2,3,3-Tetrafluoro-3-{[1,1,1,2,3,3-hexafluoro...",773804-62-9


In [17]:
def checkVariablesT1(dfx):
    try:
        conn = sqlite3.connect('../RoundOneDB/RoundOneDB_final.db')
        cur = conn.cursor()

        df = pd.DataFrame(columns = ['analyte_name', 'abbreviation', 'cas', 'class']) 
        cur.execute("""SELECT DISTINCT analyte_name, abbreviation, cas, class FROM quant_summary 
                       WHERE cas = '749836-20-2'""")
        row = cur.fetchall()
        df = df.append({'analyte_name': row[0][0],'abbreviation': row[0][1],'cas': row[0][2], 
                        'class': row[0][3]}, ignore_index = True)
        cur.execute("""SELECT DISTINCT analyte_name, abbreviation, cas, class FROM quant_summary 
                       WHERE cas = '29311-67-9'""")
        row = cur.fetchall()
        df = df.append({'analyte_name': row[0][0],'abbreviation': row[0][1],'cas': row[0][2], 
                        'class': row[0][3]}, ignore_index = True)
        cur.execute("""SELECT DISTINCT analyte_name, abbreviation, cas, class FROM quant_summary 
                       WHERE cas = '267239-61-2'""")
        row = cur.fetchall()
        df = df.append({'analyte_name': row[0][0],'abbreviation': row[0][1],'cas': row[0][2], 
                        'class': row[0][3]}, ignore_index = True)
        cur.execute("""SELECT DISTINCT analyte_name, abbreviation, cas, class FROM quant_summary 
                       WHERE cas = '674-13-5'""")
        row = cur.fetchall()
        df = df.append({'analyte_name': row[0][0],'abbreviation': row[0][1],'cas': row[0][2], 
                        'class': row[0][3]}, ignore_index = True)
        cur.execute("""SELECT DISTINCT analyte_name, abbreviation, cas, class FROM quant_summary 
                       WHERE cas = '13252-13-6'""")
        row = cur.fetchall()
        df = df.append({'analyte_name': row[0][0],'abbreviation': row[0][1],'cas': row[0][2], 
                        'class': row[0][3]}, ignore_index = True)
        cur.execute("""SELECT DISTINCT analyte_name, abbreviation, cas, class FROM quant_summary 
                       WHERE cas = '773804-62-9'""")
        row = cur.fetchall()
        df = df.append({'analyte_name': row[0][0],'abbreviation': row[0][1],'cas': row[0][2], 
                        'class': row[0][3]}, ignore_index = True)

        cur.close()
        conn.close()

        return df

    except Error as e:
        print(e)

In [18]:
dfy = checkVariablesT1(dfx)
dfy

,analyte_name,abbreviation,cas,class
0,"Ethanesulfonic acid, 2-[1-[difluoro(1,2,2,2-te...",Nafion by-product 2,749836-20-2,Perfluoroalkylether Acids (PFEAs)
1,"Ethanesulfonic acid, 2-[1-[difluoro[(1,2,2-tri...",Nafion by-product 1,29311-67-9,Perfluoroalkylether Acids (PFEAs)
2,Perfluoro-2-ethoxypropanoic acid,PEPA,267239-61-2,Perfluoroalkylether Acids (PFEAs)
3,Perfluoro-2-methoxyacetic acid,PFMOAA,674-13-5,Perfluoroalkylether Acids (PFEAs)
4,Perfluoro-2-propoxypropanoic acid,GenX,13252-13-6,Perfluoroalkylether Acids (PFEAs)
5,"Propanoic acid, 3-[1-[difluoro(1,2,2,2-tetrafl...",Hydro-EVE acid,773804-62-9,Perfluoroalkylether Acids (PFEAs)


In [2]:
def getVariablesCV():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT DISTINCT v.variableid, v.variablecode, v.nodatavalue, v.variablenamecv, 
                                       cv.term, u.unitid, u.unitabbreviation, u.unitname
                       FROM results r 
                       INNER JOIN units u ON r.unitid=u.unitid 
                       INNER JOIN feature_actions fa ON r.featureactionid=fa.featureactionid 
                       INNER JOIN sampling_features sf ON fa.samplingfeatureid=sf.samplingfeatureid
                       INNER JOIN variables v ON r.variableid=v.variableid
                       INNER JOIN cv_variable_names cv ON v.variablenamecv=cv.name
                       WHERE sf.samplingfeaturecode LIKE 'GB%' OR sf.samplingfeaturecode LIKE 'MC%'""")
        rows = cur.fetchall()
        
        df = pd.DataFrame(columns = ['variableid', 'variablecode', 'nodatavalue', 'variablenamecv', 
                                     'term', 'unitid', 'unitabbreviation', 'unitname']) 
        
        for row in rows:
            df = df.append({'variableid': row[0], 'variablecode': row[1], 'nodatavalue': row[2], 
                            'variablenamecv': row[3], 'term': row[4], 'unitid': row[5], 
                            'unitabbreviation': row[6], 'unitname': row[7]}, 
                            ignore_index = True)

        return df
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [3]:
df = getVariablesCV()
df.head()

,variableid,variablecode,nodatavalue,variablenamecv,term,unitid,unitabbreviation,unitname
0,53544,PFBA,4.9,Perfluorobutanoic acid,375-22-4,2599,ng/L,nanograms per liter
1,53545,PFPeA,4.9,Perfluoropentanoic acid,2706-90-3,2599,ng/L,nanograms per liter
2,53546,PFHxA,4.9,Perfluorohexanoic acid,307-24-4,2599,ng/L,nanograms per liter
3,53547,PFOA,4.9,Perfluorooctanoic acid,335-67-1,2599,ng/L,nanograms per liter
4,53548,PFNA,4.9,Perfluorononanoic acid,375-95-1,2599,ng/L,nanograms per liter


In [4]:
len(df['unitid'])

23

In [5]:
len(np.unique(np.array(df['unitid'])))

1